In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import LeaveOneOut
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
import seaborn as sns
from sklearn.tree import export_graphviz
import sklearn.metrics as metrics
from sklearn.metrics import r2_score as r2
from sklearn.feature_selection import SequentialFeatureSelector
import re
from matplotlib.ticker import AutoLocator
import graphviz
from graphviz import render



matplotlib.style.use('ggplot')

In [ ]:
# function to remove assays and medias with less than 4 occurrences

def agg_size_nosort(df):
    counts_assay = df.groupby("assay", sort=False)["assay"].transform('size')
    counts_media = df.groupby("media", sort=False)["media"].transform('size')
    mask = (counts_assay > 3) & (counts_media > 3)
    return df[mask]

# function to plot data in bins
def plot_bar_bin(series, bins, img_name):
    '''
    plot an histogram with ranges of data as columns
    :param series: series of the feature to plot
    :param bins: a list with the limit values of the bins
    :param img_name: name of the saved figure
    :return: histogram of data points in each bin
    '''
    # general bar plots with 6 bins
    to_plot = series.dropna(axis=0)
    color = cm.rainbow(np.linspace(0, 1, len(bins) - 1))
    fig, ax = plt.subplots()
    hist, bin_edges = np.histogram(to_plot, bins)
    ax.set_ylim(0, 50)
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_ylabel('Number of data points')
    ax.set_xlabel('BMDL range [ug/mL]')
    rec = ax.bar(range(len(hist)), hist, width=0.8, align='center', tick_label=
    ['{} - {}'.format(bins[i], bins[i + 1]) for i, j in enumerate(hist)], color=color)
    plt.xticks(rotation=45)
    autolabel(rec, ax)
    plt.tight_layout()
    plt.savefig(f'{img_name}.png')
    plt.show()

def autolabel(rects, ax):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2., 1.05 * height,
               s= '%d' % int(height),
                ha='center', va='bottom')


#Function to print positive feature importance based on the Linear SVC coefficients, per class

def print_top10(feature_names, clf, x_name, class_labels):
    """Prints features with the highest coefficient values, per class"""
    for i, class_label in enumerate(class_labels):
        coeff = clf.coef_[i]
        ordered_coef = [x for x,_ in sorted(zip(coeff,feature_names), key=lambda sublist: abs(sublist[0]))[::-1]]
        ordered_names = [y for _,y in sorted(zip(coeff,feature_names), key=lambda sublist: abs(sublist[0]))[::-1]]
        ordered_coef_pos = [x for x in ordered_coef[0:10]]
        ordered_names_pos = ordered_names[:len(ordered_coef_pos)]

        fig = plt.figure()
        plt.barh(range(len(ordered_names_pos)), ordered_coef_pos[::-1], align='center')
        plt.yticks(range(len(ordered_names_pos)), ordered_names_pos[::-1])

        plt.title(f'Feature importance for class {class_label}')
        plt.tight_layout()
        #plt.savefig(f'Graphene//New_plots//BMDL LinearSVC viab 3class coeff2 {x_name} {i}.pdf')
        plt.show()


# Function to plot confusion matrix

def confusion_matrix(matrix, tick_names, file_name, data_set_name):
        # Plot confusion matrix
        figure = plt.figure()
        ax = plt.subplot()
        im = ax.imshow(matrix, cmap=plt.cm.Blues, interpolation='nearest')
        #tick_marks = [0,1,2,3]
        tick_marks=[0, 1, 2]
        ax.set_xticks(tick_marks)
        ax.set_xticklabels(tick_names)
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
        plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
                 rotation_mode="anchor", fontsize=12)
        ax.set_yticks(tick_marks)
        ax.set_yticklabels(tick_names)
        plt.setp(ax.get_yticklabels(), fontsize=12)
        ax.set_ylim(-0.5, 2.5)
        # change font color to white to be visible against darkest colors
        for (i, j), z in np.ndenumerate(matrix):
            if z > 8:
                fontcolor = 'white'
            else:
                fontcolor = 'black'
            ax.text(j, i, str(int(z)), ha='center', va='center', color= fontcolor, size=12)
        fig.colorbar(im, ax=ax)
        ax.grid(None)
        plt.tight_layout()
        plt.savefig(f'{file_name}_{data_set_name}.pdf', dpi=400)
        plt.savefig(f'{file_name}_{data_set_name}.png', dpi=400)
        plt.show()

In [ ]:
################################
#   LOADING DATA
#################################
df_v = pd.read_csv('Graphene/Graphene_BMD_UPDATED_no_outliers.csv')
df_v.describe()



In [ ]:
################################
#   PREPARING DATA FOR ANALYSIS
#################################

df_v[['new_name', 'species']] = df_v[['new_name', 'species']].applymap(lambda x: x.strip())
df_v['func'] = df_v['func'].fillna(0)

# keep only alpah numeric characters in media column
df_v['media'] = df_v['media'].apply(lambda row: re.sub(r'\W+', '', row))

df_v['assay'] = df_v['assay'].apply(lambda row: row.replace('viability_', ''))

print(f'Before removing less than 4 assay and media, size = {len(df_v)}')

df_v = agg_size_nosort(df_v)
print(f'After removing less than 4 assay and media, size = {len(df_v)}')


In [ ]:
############################################
## SUPERVISED CLASSIFICATION
############################################

In [ ]:
bins = [0, 5, 10, 15, 30, 50, 70, 100, 120, 140, 260]
plot_bar_bin(df_v['BMDL'], bins, 'BMDL distribution - viability')

In [ ]:
# Define BMDL classes
df_v['BMDL_class'] = np.nan

# 4 classes
'''
df_v.loc[df_v.BMDL < 15, 'BMDL_class'] = 0
df_v.loc[(df_v.BMDL >= 15) & (df_v.BMDL < 35), 'BMDL_class'] = 1
df_v.loc[(df_v.BMDL >= 35) & (df_v.BMDL < 100), 'BMDL_class'] = 2
df_v.loc[df_v.BMDL >= 100, 'BMDL_class'] = 3
'''
# 3 classes
df_v.loc[df_v.BMDL < 15, 'BMDL_class'] = 0
df_v.loc[(df_v.BMDL >= 15) & (df_v.BMDL < 60), 'BMDL_class'] = 1
df_v.loc[df_v.BMDL >= 60, 'BMDL_class'] = 2

# Plot distribution of data in the classes
fig = plt.figure()
plt.hist(df_v.BMDL_class)
plt.show()

In [ ]:
# Data set 1

sc = df_v[['BMDL_class', 'Substance', 'size_class', 'time', 'cell_type', 'assay','func', 'layer', 'z_pot', 'media',
           'cell_species','cell_type_general', 'species']].copy()
sc.dropna(axis=0, inplace=True)
sc.reset_index(drop=True, inplace=True)
# categorical variables encoding
enc_size = {'S':1, 'M':2, 'L':3}
sc['size_class'] = sc['size_class'].apply(lambda row: enc_size[row])
sc = pd.get_dummies(sc, columns=['Substance',  'cell_type', 'cell_species', 'cell_type_general', 'species' , 'assay',
                                 'media', 'func'], drop_first=True)

y_1 = sc['BMDL_class']
x_source_1 = sc.drop(labels='BMDL_class', axis=1)

# plot distribution of dependent variable
fig = plt.figure()
plt.hist(y_1)
plt.show()

In [ ]:
# Data set 2

sc2 = df_v[['BMDL_class', 'Substance', 'size_class', 'time', 'cell_type', 'assay','func',  'cell_species', 'media',
            'cell_type_general', 'species']].copy()

enc_size = {'S':1, 'M':2, 'L':3}
sc2.dropna(axis=0, inplace=True)
sc2.reset_index(drop=True, inplace=True)
# categorical variables encoding
sc2['size_class'] = sc2['size_class'].apply(lambda row: enc_size[row])
sc2 = pd.get_dummies(sc2, columns=['Substance',  'cell_type', 'cell_species', 'cell_type_general', 'species' , 'assay',
                                   'func', 'media'], drop_first=True)

y_big = sc2['BMDL_class']
x_source_big = sc2.drop(labels='BMDL_class', axis=1)

fig = plt.figure()
plt.hist(y_big)
plt.show()

In [ ]:
# Data set 3

sc3 = df_v[['BMDL_class', 'Substance', 'size_class', 'time', 'cell_type', 'assay','func',  'cell_species', 'media',
            'cell_type_general', 'species', 'layer']].copy()

enc_size = {'S':1, 'M':2, 'L':3}
sc3.dropna(axis=0, inplace=True)
sc3.reset_index(drop=True, inplace=True)
# categorical variables encoding
sc3['size_class'] = sc3['size_class'].apply(lambda row: enc_size[row])
sc3 = pd.get_dummies(sc3, columns=['Substance',  'cell_type', 'cell_species', 'cell_type_general', 'species' , 'assay',
                                   'func', 'media'], drop_first=True)

y_layer = sc3['BMDL_class']
x_source_l = sc3.drop(labels='BMDL_class', axis=1)

fig = plt.figure()
plt.hist(y_layer)
plt.show()

In [ ]:
# Run this to select data set 1

y = y_1
x_source = x_source_1

In [ ]:
# Run this to select data set 2

y = y_big
x_source = x_source_big

In [ ]:
# Run this to select data set 3

y = y_layer
x_source = x_source_l

In [ ]:
x_source.columns

In [ ]:
##############################
# SVM classifier
#############################

In [ ]:
#

# pipeline that scales the data and fits a Linear SVC
pipe = Pipeline([('scaler', MinMaxScaler()), ('SVC', LinearSVC(C=1, loss='hinge', max_iter=100000))])

X = x_source
# keep track of selected parameters and corresponding model R2
parameters = []
score = []

# Sequential feature selector
for n_param in range(2,len(X.columns)):
    sfs = SequentialFeatureSelector(estimator=pipe, n_features_to_select=n_param)
    sfs.fit(X, y)
    new_x = sfs.transform(X)
    parameters.extend([sfs.get_support()])
    # cross validation - one R2 for each cross val run
    score.append(cross_val_score(pipe, new_x, y, cv=LeaveOneOut()))

# for each number of parameters tested report the R2 of Cross validation as the mean of each run
score_synt = [np.mean(x) for x in score]
print(f'SVC Model prediction (LOOCV) Accuracy for 2 to {len(X.columns)-1} features included: {score_synt}')


In [ ]:
# extract multiple sets of independent variables based on the results of feature selection by choosing the highest score_synt and selecting the corresponding position in parameters

# use also all features (x_tot)

# depending on the data set used the sets change - comment the ones that are not used.

x_68 = X.loc[:, parameters[4]].copy()
'''
x_tot = X[['size_class', 'time', 'layer', 'z_pot', 'Substance_Graphene oxide',
       'Substance_rGO', 'cell_type_BEAS- 2B', 'cell_type_J774.A1',
       'cell_type_THP-1', 'cell_species_macrophage_human',
       'cell_species_macrophage_rodent', 'cell_type_general_macrophage',
       'species_rodent', 'assay_mtt', 'assay_pi', 'assay_wst',
       'media_DMEM10FBS', 'media_DMEMF1210FBS', 'media_F1210FBS',
       'media_RPMI10FBS', 'func_NH2', 'func_PAA', 'func_PAM', 'func_PEG',
       'func_hydrated']]



# FOR LARGE DATASET
x_14 = X.loc[:, parameters[16]].copy()
x_tot_big = X[['size_class', 'time', 'Substance_Graphene oxide', 'Substance_rGO',
       'cell_type_BEAS- 2B', 'cell_type_J774.A1', 'cell_type_THP-1',
       'cell_species_macrophage_human', 'cell_species_macrophage_rodent',
       'cell_type_general_macrophage', 'species_rodent', 'assay_ez_cyto',
       'assay_ldh', 'assay_mtt', 'assay_pi', 'assay_wst', 'func_COOH',
       'func_Fe3O4', 'func_NH2', 'func_PAA', 'func_PAM', 'func_PEG',
       'func_hydrated', 'media_DMEM10FBS', 'media_DMEMF1210FBS',
       'media_F1210FBS', 'media_RPMI10FBS']]
'''
# For layer dataset

x_tot_l = X[['size_class', 'time', 'layer', 'Substance_Graphene oxide',
       'Substance_rGO', 'cell_type_BEAS- 2B', 'cell_type_J774.A1',
       'cell_type_THP-1', 'cell_species_macrophage_human',
       'cell_species_macrophage_rodent', 'cell_type_general_macrophage',
       'species_rodent', 'assay_ez_cyto', 'assay_ldh', 'assay_mtt', 'assay_pi',
       'assay_wst', 'func_COOH', 'func_NH2', 'func_PAA', 'func_PAM',
       'func_PEG', 'func_hydrated', 'media_DMEM10FBS', 'media_DMEMF1210FBS',
       'media_F1210FBS', 'media_RPMI10FBS']]

x_68l = X.loc[:, parameters[22]].copy()

In [ ]:
# choose which set of independent variables to use
x_to_use = x_tot_l
# name to use in the figures to save
x_name = 'all_61_l'

x_to_use.columns

In [ ]:
# Tune the parameters via GridSearcCV
pipe = Pipeline([('scaler', MinMaxScaler()), ('SVC', LinearSVC(loss='hinge', max_iter=10000000))])
search_space = [{'SVC__C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}]
pipe_svc = GridSearchCV(pipe, search_space, cv=LeaveOneOut())
pipe_svc.fit(x_to_use, y)

# print best C value and R2
best_C = pipe_svc.best_params_
print(best_C)
print(pipe_svc.best_score_)

In [ ]:
# print top 10 features
sns.set_style('whitegrid')

feature_names = [x for x in x_to_use.columns]
#class_names = ['BMDL < 12', '12<=BMDL<25', '25<=BMDL<50', 'BMDL>=50']
class_names = ['BMDL < 15', '15<=BMDL<60', 'BMDL>=60']
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(x_to_use)
svm = LinearSVC(C=best_C['SVC__C'], loss='hinge', max_iter=10000000)
svm.fit(X_norm, y)

print_top10(feature_names, svm, x_name, class_names)

In [ ]:
# Final model, confusion matrix and performance metics

matplotlib.style.use('ggplot')
# and get accuracy measures
pipe_svc2 = Pipeline([('scaler', MinMaxScaler()), ('SVC', LinearSVC(C=best_C['SVC__C'], loss='hinge',  max_iter=100000))])
loo = LeaveOneOut()
y_true = []
y_pred = []

x_source2 = np.array(x_to_use)
#Fit model and get predictions via LOOCV
for train_index, test_index in loo.split(x_source2):
    X_train, X_test = x_source2[train_index], x_source2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pipe_svc2.fit(X_train, y_train)
    preds = pipe_svc2.predict(X_test)
    y_true.extend(y_test)
    y_pred.extend(preds)

# Accuracy score
accuracy_avg = metrics.accuracy_score(y_true, y_pred)

# Confusion matrix
confusion = metrics.confusion_matrix(y, y_pred)
#tick_names = ['BMDL < 12', '12<=BMDL<25', '25<=BMDL<50', 'BMDL>=50']
tick_names = ['BMDL < 15', '15<=BMDL<60', 'BMDL>=60']

confusion_matrix(confusion, 'Graphene//New_plots//SVC_confusion_matrix_3class', x_name )
# Multiple performance metrics
recap = metrics.classification_report(y_true, y_pred, target_names=tick_names)
print(recap)

In [ ]:
###############################
# Decision tree classifier
###############################

In [ ]:
# select data set - change accordingly

y = y_layer
x_source = x_source_l
x_source.columns
x_name = 'all_l_61'

In [ ]:
# Find optimal max depth and min sample split via GridSearchCV
tree_class = DecisionTreeClassifier(class_weight='balanced')
search_space = [{'max_depth': [x for x in range(3,6)], 'min_samples_split': [x for x in range(2, 6)]}]
tree_res = GridSearchCV(tree_class, search_space, cv=LeaveOneOut())
tree_res.fit(x_source, y)
print(tree_res.best_params_)
print(tree_res.best_score_)

In [ ]:
# Input best parameters from above and fit model
tree_class2 = DecisionTreeClassifier(class_weight='balanced', max_depth=tree_res.best_params_['max_depth'], min_samples_split=tree_res.best_params_['min_samples_split'])
loo = LeaveOneOut()
y_true = []
y_pred = []

x_source2 = np.array(x_source)
for train_index, test_index in loo.split(x_source2):
    X_train, X_test = x_source2[train_index], x_source2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tree_class2.fit(X_train, y_train)
    preds = tree_class2.predict(X_test)
    y_true.extend(y_test)
    y_pred.extend(preds)

accuracy_avg = metrics.accuracy_score(y_true, y_pred)
print(accuracy_avg)

In [ ]:
# confusion matrix
confusion = metrics.confusion_matrix(y, y_pred)
#tick_names = ['BMDL < 12', '12<=BMDL<25', '25<=BMDL<50', 'BMDL>=50']
tick_names = ['BMDL < 15', '15<=BMDL<60',  'BMDL>=60']
confusion_matrix(confusion, 'Graphene//New_plots//Decision_tree_confusion_matrix_3class', x_name )
# performance metrics
recap = metrics.classification_report(y_true, y_pred, target_names=tick_names)
print(recap)

In [ ]:
# Draw tree
feature_names = x_source.columns
dot_data = export_graphviz(tree_res.best_estimator_, out_file=f'Graphene//New_plots//Decision_tree_classifier_3class_cat_15-60_balanced_{x_name}.dot',
            filled=True, rounded=True, feature_names=feature_names, class_names=['BMDL < 15', '15<=BMDL<60',  'BMDL>=60'])

render('dot', 'png', f'Graphene//New_plots//Decision_tree_classifier_3class_cat_15-60_balanced_{x_name}.dot')


In [ ]:
##############################
# Random Forest classifier
##############################

In [ ]:
# Find optimal max depth and min sample split via GridSearchCV
forest_class = RandomForestClassifier()
search_space = [{'n_estimators': [10], 'max_depth': [x for x in range(3,6)], 'min_samples_split': [x for x in range(2, 6)]}]
forest_res = GridSearchCV(forest_class, search_space, cv=LeaveOneOut())
forest_res.fit(x_source, y)
print(forest_res.best_params_)
print(forest_res.best_score_)

In [ ]:
# Input best parameters from above
forest_class2 = RandomForestClassifier(n_estimators=10, max_depth=forest_res.best_params_['max_depth'], min_samples_split=forest_res.best_params_['min_samples_split'])
loo = LeaveOneOut()
y_true = []
y_pred = []

x_source2 = np.array(x_source)
for train_index, test_index in loo.split(x_source2):
    X_train, X_test = x_source2[train_index], x_source2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    forest_class2.fit(X_train, y_train)
    preds = forest_class2.predict(X_test)
    y_true.extend(y_test)
    y_pred.extend(preds)


# define the model
model = RandomForestClassifier()
# evaluate the model
cv = LeaveOneOut()
n_scores = cross_val_score(forest_class2, x_source2, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
accuracy_avg = metrics.balanced_accuracy_score(y_true, y_pred)
print(accuracy_avg)

In [ ]:
# Confusion matrix
confusion = metrics.confusion_matrix(y, y_pred)
#tick_names = ['BMDL < 12', '12<=BMDL<25', '25<=BMDL<50', 'BMDL>=50']
tick_names = ['BMDL < 15', '15<=BMDL<60',  'BMDL>=60']
confusion_matrix(confusion, 'Graphene//New_plots//Random_forest_confusion_matrix_3class', x_name )